In [53]:
import json
import pandas as pd
import matplotlib.pyplot as plt

In [54]:
# Read the CSV with latin1 encoding to avoid UnicodeDecodeError
df = pd.read_csv("../../scraper/assets/data/listings.csv", encoding="latin1")

# Fix garbled text (mojibake) in all string columns
for col in df.select_dtypes(include='object'):
    df[col] = df[col].apply(lambda x: x.encode('latin1').decode('utf-8') if isinstance(x, str) else x)



In [55]:
def try_fix_unicode(text):
    if isinstance(text, str) and '\\u' in text:
        try:
            return text.encode('utf-8').decode('unicode_escape')
        except Exception:
            return text  # if decoding fails, return original
    else:
        return text

for col in df.select_dtypes(include='object'):
    df[col] = df[col].apply(try_fix_unicode)

In [56]:
df.description[4]

'"Las imágenes del apartamento presentan un estilo arquitectónico moderno con acabados de alta calidad. Los materiales utilizados incluyen madera en los pisos y tonos neutros en las paredes, que aportan un aspecto elegante y minimalista. El mobiliario visible es escaso y de diseño sencillo, sugiriendo un enfoque contemporáneo. En cuanto a iluminación natural, hay grandes ventanales que abarcan del piso al techo, permitiendo una abundante entrada de luz, principalmente en la sala de estar que cuenta con orientación hacia una fachada urbana. La distribución del espacio es abierta, como se observa en la integración de la sala con la cocina mediante una barra que hace las veces de división. Las habitaciones parecen contar también con buena entrada de luz y vistas al entorno urbano que se muestra como cierre y con edificaciones aledañas similares. En el baño, los diseños son modernos, con acabados en porcelana y detalles en vidrio. El estado general del apartamento es de excelente conservac

In [ ]:
df['facilities'] = df['facilities'].apply(json.loads)
df['technical_data'] = df['technical_data'].apply(json.loads)
df['coordinates'] = df['coordinates'].apply(json.loads)
df['coordinates'] = df['coordinates'].apply(tuple)
df['Price'] = df['Price'] + df['administracion']


In [58]:
# Normalize the dictionaries into a DataFrame
td_df = pd.json_normalize(df['technical_data'])

# Add prefix 'td_' to the new columns
td_df = td_df.add_prefix('td_')

# Concatenate the original df with the new expanded columns
df_expanded = pd.concat([df.drop(columns=['technical_data']), td_df], axis=1)

In [ ]:
df_expanded['td_Estrato'] = df_expanded['td_Estrato'].astype(int)

In [63]:
df_expanded['td_Parqueaderos'] = df_expanded['td_Parqueaderos'].fillna(0)
df_expanded['td_Parqueaderos'] = df_expanded['td_Parqueaderos'].astype(int)


In [ ]:
df_expanded['td_Piso N°'] = df_expanded['td_Piso N°'].fillna(-1)
df_expanded['td_Piso N°'] = df_expanded['td_Piso N°'].astype(int)

In [59]:
df_expanded.isna().sum()

Link                      0
Price                   745
Bedrooms                 25
Bathrooms                25
Area                     25
Agency                    0
Location                  0
Datetime_Added            0
coordinates               0
administracion          745
facilities                0
upload_date               0
description               0
places                    0
td_Estrato                1
td_Tipo de Inmueble       0
td_Estado              1095
td_Baños                  3
td_Área Construida        0
td_Área Privada         283
td_Antigüedad            70
td_Habitaciones           3
td_Parqueaderos         453
td_Administración       745
td_Piso N°              486
td_Pisos interiores     948
dtype: int64

In [47]:
df_expanded['td_Antigüedad'].value_counts()

td_Antigüedad
1 a 8 años        336
16 a 30 años      269
9 a 15 años       239
más de 30 años    184
menor a 1 año     116
Name: count, dtype: int64

In [60]:
df_expanded['td_Parqueaderos'].value_counts()

td_Parqueaderos
1    433
2    251
3     55
4     21
5      1
Name: count, dtype: int64

In [61]:
df_expanded['td_Piso N°'].value_counts()

td_Piso N°
2     128
3     117
4      95
5      94
1      88
6      64
7      33
8      21
18     16
10     13
12     12
11      9
13      9
14      9
15      7
16      7
9       6
Name: count, dtype: int64

In [62]:
df_expanded.head(20)

,Link,Price,Bedrooms,Bathrooms,Area,Agency,Location,Datetime_Added,coordinates,administracion,...,td_Estado,td_Baños,td_Área Construida,td_Área Privada,td_Antigüedad,td_Habitaciones,td_Parqueaderos,td_Administración,td_Piso N°,td_Pisos interiores
0,https://www.fincaraiz.com.co/apartamento-en-ar...,2500000.0,1.0,1.0,42.0,Houm,"Bogotá, Bogotá, d.c.",2025-05-18 13:29:56,"(4.6191593989004005, -74.06516790390015)",500000.0,...,Excelente estado,1,42.00 m2,42.00 m2,1 a 8 años,1,1,"500,000.00",NaN,NaN
1,https://www.fincaraiz.com.co/apartamento-en-ar...,3000000.0,3.0,2.0,61.0,RV Inmobiliaria,"Bogotá, Bogotá, d.c.",2025-05-18 13:30:26,"(4.656523208947739, -74.12344694137573)",320000.0,...,NaN,2,61.00 m2,61.00 m2,1 a 8 años,3,1,"320,000.00",NaN,NaN
2,https://www.fincaraiz.com.co/apartamento-en-ar...,NaN,2.0,1.0,45.0,Carlos,"Bogotá, Bogotá, d.c.",2025-05-18 13:30:52,"(4.778759707688192, -74.04969692230225)",NaN,...,NaN,1,45.00 m2,NaN,menor a 1 año,2,NaN,NaN,11,11
3,https://www.fincaraiz.com.co/apartamento-en-ar...,NaN,3.0,2.0,60.0,Fanny ConsueloHernandez,"Bogotá, Bogotá, d.c.",2025-05-18 13:31:19,"(4.630345222945569, -74.1381025314331)",NaN,...,NaN,2,60.00 m2,60.00 m2,más de 30 años,3,1,NaN,4,NaN
4,https://www.fincaraiz.com.co/apartamento-en-ar...,4365403.0,1.0,2.0,60.0,Engel & Völkers,"Bogotá, Bogotá, d.c.",2025-05-18 13:31:46,"(4.6769473501863805, -74.05478239059448)",765403.0,...,NaN,2,60.00 m2,NaN,9 a 15 años,1,2,"765,403.00",1,1
5,https://www.fincaraiz.com.co/apartamento-en-ar...,900000.0,3.0,1.0,47.0,RV Inmobiliaria,"Bogotá, Bogotá, d.c.",2025-05-18 13:32:12,"(4.624420821888976, -74.20773267745972)",65000.0,...,NaN,1,47.00 m2,47.00 m2,1 a 8 años,3,NaN,"65,000.00",NaN,NaN
6,https://www.fincaraiz.com.co/apartamento-en-ar...,1300000.0,3.0,2.0,36.0,RV Inmobiliaria,"Bogotá, Bogotá, d.c.",2025-05-18 13:32:35,"(4.6878970111815645, -74.15269374847412)",138000.0,...,NaN,2,36.00 m2,36.00 m2,1 a 8 años,3,NaN,"138,000.00",NaN,NaN
7,https://www.fincaraiz.com.co/apartamento-en-ar...,NaN,1.0,2.0,145.0,Engel & Völkers,"Bogotá, Bogotá, d.c.",2025-05-18 13:33:00,"(4.656480435253913, -74.04716491699219)",NaN,...,NaN,2,145.00 m2,NaN,más de 30 años,1,2,NaN,4,NaN
8,https://www.fincaraiz.com.co/apartamento-en-ar...,2878303.0,2.0,2.0,73.0,Gestiones Inmobiliarias F. T. SAS,"Bogotá, Bogotá, d.c.",2025-05-18 13:33:35,"(4.6471984822145345, -74.0786862373352)",428303.0,...,NaN,2,73.00 m2,73.00 m2,16 a 30 años,2,1,"428,303.00",3,NaN
9,https://www.fincaraiz.com.co/apartamento-en-ar...,850000.0,2.0,1.0,44.0,RV Inmobiliaria,"Bogotá, Bogotá, d.c.",2025-05-18 13:34:02,"(4.635478237899357, -74.1876482963562)",65000.0,...,NaN,1,44.00 m2,44.00 m2,1 a 8 años,2,NaN,"65,000.00",NaN,NaN
